In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [2]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [3]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [4]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [5]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

readInJson(garbageCollectionDate[2]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_4f2d6646_d246_433c_b537_ed0b09273cbb.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'rub_sched': 'R/2015-01-07/P2W', 'rec_sched':..."
1,Feature,ckan_4f2d6646_d246_433c_b537_ed0b09273cbb.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'rub_sched': 'R/2015-01-01/P2W', 'rec_sched':..."


In [6]:
df = readInJson(garbageCollectionDate[2])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [7]:
newCols = ['landfill_colDay', 
           'recyclable_colDay',
          'green_colDay',]
newColsKeys = ['rub_sched', 
               'rec_sched', 
              'grn_sched']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    df[key] = df.properties.apply(lambda x: x[val])

for i in ['landfill', 'recyclable', 'green']: 
    df[f'{i}_date'] = df[f'{i}_colDay'].apply(lambda x: x.split('/')[1])
    df[f'{i}_date'] = pd.to_datetime(df[f'{i}_date'], format='%Y-%m-%d').dt.day_name()
    df[f'{i}_freq'] = df[f'{i}_colDay'].apply(lambda x: x.split('/')[-1])
    df[f'{i}_freq'] = df[f'{i}_freq'].apply(lambda x: 'Every Second ' if x == 'P2W' else 'Every ')
    
    df[f'{i}_colDay'] = df[f'{i}_freq'] + df[f'{i}_date']
    
    df.drop([f'{i}_freq', f'{i}_date'], axis = 1, inplace = True)

df.drop('properties', axis = 1, inplace = True)

In [11]:
geelong = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay', 'green_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay', 'green_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay', 'green_colDay'], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})

geelong.to_csv('../GBD_geelong.csv', index = False)